In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_avcdmdmm152',
    proc_date,
    'temp_sap_avcdmdmm152'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm152
(
  kschg STRING,
  spras STRING,
  klpos STRING,
  klart STRING,
  class STRING,
  clint STRING,
  recordmode STRING,
  proc_date DATE,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm152
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avcdmdmm152
SELECT 
        {catalog_name}.default.fn_CLEAN_SAP_STRING(kschg) AS kschg,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(spras) AS spras,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(klpos) AS klpos,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(klart) AS klart,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(class) AS class,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(clint) AS clint,
        {catalog_name}.default.fn_CLEAN_SAP_STRING(recordmode) AS recordmode,
        proc_date,
        file_creation_ts
    FROM temp_sap_avcdmdmm152
""")